In [6]:
import pandas as pd
df = pd.read_csv("C:\\Users\\MD USMAN ANSARI\\BO\\Spotify Million Song Dataset_exported.csv")
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [7]:
# Inspect the first few rows and columns to understand the dataset structure
print(df.head())
print(df.columns)

  artist                   song                                        link  \
0   ABBA  Ahe's My Kind Of Girl  /a/abba/ahes+my+kind+of+girl_20598417.html   
1   ABBA       Andante, Andante       /a/abba/andante+andante_20002708.html   
2   ABBA         As Good As New        /a/abba/as+good+as+new_20003033.html   
3   ABBA                   Bang                  /a/abba/bang_20598415.html   
4   ABBA       Bang-A-Boomerang      /a/abba/bang+a+boomerang_20002668.html   

                                                text  
0  Look at her face, it's a wonderful face  \nAnd...  
1  Take it easy with me, please  \nTouch me gentl...  
2  I'll never know why I had to go  \nWhy I had t...  
3  Making somebody happy is a question of give an...  
4  Making somebody happy is a question of give an...  
Index(['artist', 'song', 'link', 'text'], dtype='object')


In [8]:
# Text preprocessing: tokenization and stop-word removal
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize
    tokens = text.split()
    # Remove stop words
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return ' '.join(tokens)

# Apply preprocessing to the lyrics/text column
df['processed_text'] = df['text'].astype(str).apply(preprocess_text)
df[['text', 'processed_text']].head()

,text,processed_text
0,"Look at her face, it's a wonderful face \nAnd...",look face wonderful face means special look wa...
1,"Take it easy with me, please \nTouch me gentl...",easy touch gently like summer evening breeze t...
2,I'll never know why I had to go \nWhy I had t...,ill know lousy rotten boy tough packing stuff ...
3,Making somebody happy is a question of give an...,making somebody happy question learn come brea...
4,Making somebody happy is a question of give an...,making somebody happy question learn come brea...


In [9]:
# Build a simple text similarity model using TF-IDF and Nearest Neighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Vectorize the processed lyrics
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_text'])

# Fit Nearest Neighbors model
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(X)

def identify_song_artist(snippet):
    # Preprocess the input snippet
    processed = preprocess_text(snippet)
    # Vectorize
    vec = vectorizer.transform([processed])
    # Find nearest neighbor
    dist, idx = nn.kneighbors(vec)
    song = df.iloc[idx[0][0]]['song']
    artist = df.iloc[idx[0][0]]['artist']
    return song, artist, dist[0][0]

# Example usage (will be tested in the next cell):
# identify_song_artist("Making somebody happy is a question of give and take")

In [12]:
# Test the model with a sample lyrics snippet
snippet = input("Enter a lyrics snippet to identify the song and artist: ")
song, artist, distance = identify_song_artist(snippet)
print(f"Predicted Song: {song}\nPredicted Artist: {artist}\nSimilarity Distance: {distance}")

Predicted Song: Poprocks And Coke
Predicted Artist: Green Day
Similarity Distance: 0.211084347273089
